In [7]:
import sys
print(sys.executable)

/home/disk/p/wycheng/anaconda3/envs/incubator2021/bin/python


In [8]:
import xarray as xr
import dask
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from eofs.xarray import Eof
import time

# Read data

In [9]:
WWLLN_dataset = xr.open_mfdataset('/home/disk/eos12/wycheng/data/US/WWLLN/WWLLN_*_F_cg_1deg3hr_US.nc',
                                  parallel=True,
                                  chunks={'Time':'auto', 'lat':'auto', 'lon':'auto'})
WWLLN_dataset['F'] = (1/((111.19492664455873)**2)) * (365.25*8) * WWLLN_dataset['F']
display(WWLLN_dataset)

<xarray.Dataset>
Dimensions:  (Time: 29216, lat: 40, lon: 75)
Coordinates:
  * Time     (Time) datetime64[ns] 2010-01-01 ... 2019-12-31T21:00:00
  * lat      (lat) float32 20.5 21.5 22.5 23.5 24.5 ... 55.5 56.5 57.5 58.5 59.5
  * lon      (lon) float32 -134.5 -133.5 -132.5 -131.5 ... -62.5 -61.5 -60.5
Data variables:
    F        (Time, lat, lon) float32 dask.array<chunksize=(2920, 40, 75), meta=np.ndarray>
Attributes:
    creation_date:  Sat Mar  6 22:09:14 PST 2021
    Conventions:    None
    title:          level 1 analysis

In [10]:
TRMM_dataset = xr.open_mfdataset('/home/disk/eos12/wycheng/data/US/TRMM/TRMM_*_pcp_cg_1deg3hr_US.nc',
                                  parallel=True,
                                  chunks={'Time':'auto', 'lat':'auto', 'lon':'auto'})
display(TRMM_dataset)

<xarray.Dataset>
Dimensions:  (Time: 29216, lat: 40, lon: 75)
Coordinates:
  * Time     (Time) datetime64[ns] 2010-01-01 ... 2019-12-31T21:00:00
  * lat      (lat) float32 20.5 21.5 22.5 23.5 24.5 ... 55.5 56.5 57.5 58.5 59.5
  * lon      (lon) float32 -134.5 -133.5 -132.5 -131.5 ... -62.5 -61.5 -60.5
Data variables:
    pcp      (Time, lat, lon) float32 dask.array<chunksize=(2920, 40, 75), meta=np.ndarray>

In [ ]:
ERA5_cape_dataset = xr.open_mfdataset('/home/disk/eos12/wycheng/data/US/ERA5/ERA5_cape_*.nc',
                                     parallel=True,
                                     chunks={'time':'auto', 'latitude':'auto', 'longitude':'auto'})

ERA5_cape_dataset = ERA5_cape_dataset.rename({'time':'Time', 'latitude':'lat', 'longitude':'lon'})

ERA5_cape_dataset = ERA5_cape_dataset.resample(Time="3h").mean()

lono = xr.DataArray(np.linspace(-134.5,-60.5,75), dims='lon')
lato = xr.DataArray(np.linspace(20.5,59.5,40), dims='lat')

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ERA5_cape_dataset = ERA5_cape_dataset.interp(lon=lono,lat=lato,method='linear')

display(ERA5_cape_dataset)

In [ ]:
ERA5_cape_dataset.to_netcdf(path='/home/disk/eos12/wycheng/data/US/ERA5/dataset/ERA5_cape_dataset.nc', mode='w')

In [ ]:
ERA5_t_dataset = xr.open_mfdataset('/home/disk/eos12/wycheng/data/US/ERA5/ERA5_t_*.nc',
                                     parallel=True,
                                     chunks={'time':'auto', 'level':'auto', 'latitude':'auto', 'longitude':'auto'}).compute()

ERA5_t_dataset = ERA5_t_dataset.rename({'time':'Time', 'level':'lev', 'latitude':'lat', 'longitude':'lon'})
ERA5_t_dataset = ERA5_t_dataset.reindex(lev=list(reversed(ERA5_t_dataset['lev'])))
ERA5_t_dataset = ERA5_t_dataset.resample(Time="3h").mean()

lono = xr.DataArray(np.linspace(-134.5,-60.5,75), dims='lon')
lato = xr.DataArray(np.linspace(20.5,59.5,40), dims='lat')

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ERA5_t_dataset = ERA5_t_dataset.interp(lon=lono,lat=lato,method='linear')

display(ERA5_t_dataset)

In [ ]:
ERA5_t_dataset.to_netcdf(path='/home/disk/eos12/wycheng/data/US/ERA5/dataset/ERA5_t_dataset.nc', mode='w')

In [11]:
ERA5_q_dataset = xr.open_mfdataset('/home/disk/eos12/wycheng/data/US/ERA5/ERA5_q_*.nc',
                                     parallel=True,
                                     chunks={'time':'auto', 'level':'auto', 'latitude':'auto', 'longitude':'auto'}).compute()

ERA5_q_dataset = ERA5_q_dataset.rename({'time':'Time', 'level':'lev', 'latitude':'lat', 'longitude':'lon'})
ERA5_q_dataset = ERA5_q_dataset.reindex(lev=list(reversed(ERA5_q_dataset['lev'])))
ERA5_q_dataset = ERA5_q_dataset.resample(Time="3h").mean()

lono = xr.DataArray(np.linspace(-134.5,-60.5,75), dims='lon')
lato = xr.DataArray(np.linspace(20.5,59.5,40), dims='lat')

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ERA5_q_dataset = ERA5_q_dataset.interp(lon=lono,lat=lato,method='linear')

display(ERA5_q_dataset)

<xarray.Dataset>
Dimensions:  (Time: 29216, lat: 40, lev: 10, lon: 75)
Coordinates:
  * Time     (Time) datetime64[ns] 2010-01-01 ... 2019-12-31T21:00:00
  * lev      (lev) int64 1000 900 800 700 600 500 400 300 200 100
  * lon      (lon) float64 -134.5 -133.5 -132.5 -131.5 ... -62.5 -61.5 -60.5
  * lat      (lat) float64 20.5 21.5 22.5 23.5 24.5 ... 55.5 56.5 57.5 58.5 59.5
Data variables:
    q        (Time, lev, lat, lon) float64 0.01065 0.01041 ... 2.779e-06

In [12]:
ERA5_q_dataset.to_netcdf(path='/home/disk/eos12/wycheng/data/US/ERA5/dataset/ERA5_q_dataset.nc', mode='w')

In [13]:
ERA5_z_dataset = xr.open_mfdataset('/home/disk/eos12/wycheng/data/US/ERA5/ERA5_z_*.nc',
                                     parallel=True,
                                     chunks={'time':'auto', 'level':'auto', 'latitude':'auto', 'longitude':'auto'}).compute()

ERA5_z_dataset = ERA5_z_dataset.rename({'time':'Time', 'level':'lev', 'latitude':'lat', 'longitude':'lon'})
ERA5_z_dataset = ERA5_z_dataset.reindex(lev=list(reversed(ERA5_z_dataset['lev'])))
ERA5_z_dataset = ERA5_z_dataset.resample(Time="3h").mean()

lono = xr.DataArray(np.linspace(-134.5,-60.5,75), dims='lon')
lato = xr.DataArray(np.linspace(20.5,59.5,40), dims='lat')

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ERA5_z_dataset = ERA5_z_dataset.interp(lon=lono,lat=lato,method='linear')

display(ERA5_z_dataset)

<xarray.Dataset>
Dimensions:  (Time: 29216, lat: 40, lev: 10, lon: 75)
Coordinates:
  * Time     (Time) datetime64[ns] 2010-01-01 ... 2019-12-31T21:00:00
  * lev      (lev) int64 1000 900 800 700 600 500 400 300 200 100
  * lon      (lon) float64 -134.5 -133.5 -132.5 -131.5 ... -62.5 -61.5 -60.5
  * lat      (lat) float64 20.5 21.5 22.5 23.5 24.5 ... 55.5 56.5 57.5 58.5 59.5
Data variables:
    z        (Time, lev, lat, lon) float64 1.455e+03 1.473e+03 ... 1.547e+05

In [16]:
ERA5_z_dataset.to_netcdf(path='/home/disk/eos12/wycheng/data/US/ERA5/dataset/ERA5_z_dataset.nc', mode='w')

In [14]:
ERA5_u_dataset = xr.open_mfdataset('/home/disk/eos12/wycheng/data/US/ERA5/ERA5_u_*.nc',
                                     parallel=True,
                                     chunks={'time':'auto', 'level':'auto', 'latitude':'auto', 'longitude':'auto'}).compute()

ERA5_u_dataset = ERA5_u_dataset.rename({'time':'Time', 'level':'lev', 'latitude':'lat', 'longitude':'lon'})
ERA5_u_dataset = ERA5_u_dataset.reindex(lev=list(reversed(ERA5_u_dataset['lev'])))
ERA5_u_dataset = ERA5_u_dataset.resample(Time="3h").mean()

lono = xr.DataArray(np.linspace(-134.5,-60.5,75), dims='lon')
lato = xr.DataArray(np.linspace(20.5,59.5,40), dims='lat')

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ERA5_u_dataset = ERA5_u_dataset.interp(lon=lono,lat=lato,method='linear')

display(ERA5_u_dataset)

<xarray.Dataset>
Dimensions:  (Time: 29216, lat: 40, lev: 10, lon: 75)
Coordinates:
  * Time     (Time) datetime64[ns] 2010-01-01 ... 2019-12-31T21:00:00
  * lev      (lev) int64 1000 900 800 700 600 500 400 300 200 100
  * lon      (lon) float64 -134.5 -133.5 -132.5 -131.5 ... -62.5 -61.5 -60.5
  * lat      (lat) float64 20.5 21.5 22.5 23.5 24.5 ... 55.5 56.5 57.5 58.5 59.5
Data variables:
    u        (Time, lev, lat, lon) float64 -11.54 -11.94 -12.28 ... 26.28 26.62

In [17]:
ERA5_u_dataset.to_netcdf(path='/home/disk/eos12/wycheng/data/US/ERA5/dataset/ERA5_u_dataset.nc', mode='w')

In [15]:
ERA5_v_dataset = xr.open_mfdataset('/home/disk/eos12/wycheng/data/US/ERA5/ERA5_v_*.nc',
                                     parallel=True,
                                     chunks={'time':'auto', 'level':'auto', 'latitude':'auto', 'longitude':'auto'}).compute()

ERA5_v_dataset = ERA5_v_dataset.rename({'time':'Time', 'level':'lev', 'latitude':'lat', 'longitude':'lon'})
ERA5_v_dataset = ERA5_v_dataset.reindex(lev=list(reversed(ERA5_v_dataset['lev'])))
ERA5_v_dataset = ERA5_v_dataset.resample(Time="3h").mean()

lono = xr.DataArray(np.linspace(-134.5,-60.5,75), dims='lon')
lato = xr.DataArray(np.linspace(20.5,59.5,40), dims='lat')

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ERA5_v_dataset = ERA5_v_dataset.interp(lon=lono,lat=lato,method='linear')

display(ERA5_v_dataset)

<xarray.Dataset>
Dimensions:  (Time: 29216, lat: 40, lev: 10, lon: 75)
Coordinates:
  * Time     (Time) datetime64[ns] 2010-01-01 ... 2019-12-31T21:00:00
  * lev      (lev) int64 1000 900 800 700 600 500 400 300 200 100
  * lon      (lon) float64 -134.5 -133.5 -132.5 -131.5 ... -62.5 -61.5 -60.5
  * lat      (lat) float64 20.5 21.5 22.5 23.5 24.5 ... 55.5 56.5 57.5 58.5 59.5
Data variables:
    v        (Time, lev, lat, lon) float64 1.423 0.8034 ... -8.444 -8.657

In [18]:
ERA5_v_dataset.to_netcdf(path='/home/disk/eos12/wycheng/data/US/ERA5/dataset/ERA5_v_dataset.nc', mode='w')